In [ ]:
%run Include.ipynb
%run FileIO.ipynb
import glob

class Data(torch.utils.data.Dataset):
    
    def __init__(self, root, root_pds, target_type, transform=None):
        self.root             = root
        self.transform        = transform
        self.address_book     = []
        self.address_book_pds = []
        os.chdir(root)
        for file in glob.glob("*."+target_type):
            self.address_book.append(os.path.join(root, file))
            self.address_book_pds.append(os.path.join(root_pds, file+".dat"))
        
    def __len__(self):
        return len(self.address_book)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img     = skio.imread(self.address_book[idx])
        pd_path = self.address_book_pds[idx]
        
        if self.transform:
            img = self.transform(img)
        
        instance = {'image': img, 'pd_path': pd_path}
        return instance
    
class Data_fetcher(object):
    
    @staticmethod
    def fetch_dataset(name, batch_size, batch_workers, shuffle, drop_last):
        if name == "cifar10":
            dataset = dset.CIFAR10(root=FLAGS.data_path, download=True,
                      transform=transforms.Compose([
                          transforms.Resize([64, 64]),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                      ]))
        elif name == "custom":
            dataset = Data(FLAGS.data_path, FLAGS.pds_path, FLAGS.data_extension,
                  transform=transforms.Compose(
                 [transforms.ToPILImage(),
                  transforms.ToTensor(),
                  transforms.Normalize([0.5], [0.5])
                 ]))
        else:
            raise NotImplementedError('Unrecognized dataset %s' % name)
            
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                     shuffle=shuffle, num_workers=int(batch_workers), drop_last=drop_last)
        return dataloader